In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def read_data():
    df = pd.read_csv('bankrupcy.csv')
    return df.to_numpy()
#convert to numpy make it faster

In [3]:
# data set to 0 - 1 impact remains the same
def normalize(data):
    minimums = np.min(data,axis=0)
    maximums =np.max(data,axis=0)
    rng= maximums-minimums
    normalize_data = 1-((maximums-data)/rng)
    return normalize_data

In [4]:
dataset = pd.read_csv('bankrupcy.csv')

dataset.describe()

,attribute1,attribute2,attribute3,attribute4,attribute5,attribute6,attribute7,attribute8,attribute9,attribute10,attribute11,attribute12,attribute13,attribute14,attribute15,attribute16,attribute17,attribute18,attribute19,attribute20,attribute21,attribute22,attribute23,attribute24,attribute25,attribute26,attribute27,attribute28,attribute29,attribute30,attribute31,attribute32,attribute33,attribute34,attribute35,attribute36,attribute37,attribute38,attribute39,attribute40,attribute41,attribute42,attribute43,attribute44,attribute45,attribute46,attribute47,attribute48,attribute49,attribute50,attribute51,attribute52,attribute53,attribute54,attribute55,attribute56,attribute57,attribute58,attribute59,attribute60,attribute61,attribute62,attribute63,attribute64,ClassLabel
count,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.00000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000,453.000000
mean,-0.079329,0.034849,-0.228204,-0.070187,0.007117,-0.070507,-0.006387,-0.137298,0.079628,-0.038179,-0.184612,-0.086773,-0.013439,-0.006392,-0.005153,-0.23991,-0.146144,-0.008432,-0.035658,0.096797,0.050098,-0.205164,-0.030013,-0.084634,-0.137836,-0.236385,-0.009777,-0.011449,-0.145535,0.044153,-0.034203,-0.009486,-0.103328,0.110358,-0.247298,-0.016978,-0.012521,-0.025538,-0.039993,-0.035146,-0.007386,-0.031183,0.012971,-0.001902,0.003613,-0.075185,-0.017667,-0.114832,-0.018169,-0.154670,0.223719,-0.010221,-0.011205,-0.011589,-0.101627,0.009393,-0.019739,-0.009330,0.022235,-0.028293,-0.005740,0.021030,-0.120022,-0.004343,0.448124
std,0.252813,0.101913,1.032602,0.302937,0.012653,1.053857,0.046343,0.839836,0.955072,0.126544,0.746761,0.380864,0.021660,0.046342,0.241928,0.78927,0.815853,0.045649,0.504346,2.210859,1.349882,0.769030,0.426189,1.633206,0.403346,0.802891,0.001834,0.068906,0.985824,1.175464,0.530826,0.063428,0.532428,0.942146,0.977401,0.959335,0.242490,0.076521,0.506831,0.251033,0.000732,0.396027,0.531838,0.205564,0.021090,0.285901,0.059416,0.677625,0.376776,0.759908,1.025234,0.062096,0.142468,0.069628,0.516665,0.009096,0.306401,0.009146,0.588701,0.069107,0.480261,0.630186,0.534885,0.205593,0.497851
min,-2.250000,-0.159000,-5.990000,-0.381000,-0.143000,-22.400000,-0.384000,-0.874000,-1.260000,-0.883000,-4.060000,-1.900000,-0.460000,-0.384000,-2.370000,-3.11000,-0.862000,-0.381000,-10.400000,-0.073400,-0.019100,-4.200000,-8.680000,-34.700000,-2.520000,-3.270000,-0.016900,-0.090400,-3.240000,-0.033900,-11.000000,-0.030700,-0.759000,-1.070000,-9.210000,-1.470000,-0.034700,-0.493000,-10.400000,-0.153000,-0.015400,-8.110000,-0.031300,-0.022300,-0.400000,-0.323000,-0.043800,-3.590000,-7.620000,-0.975000,-1.420000,-0.030100,-1.370000,-0.108000,-3.470000,-0.176000,-5.460000,-0.018200,-0.742000,-0.048600,-0.413000,-0.021100,-0.743000,-0.051000,0.000000
25%,-0.140000,-0.030600,-0.799000,-0.223000,0.006530,-0.020400,-0.020200,-0.503000,-0.396000,-0.101000,-0.443000,-0.213000,-0.012900,-0.020200,-0.029200,-0.54300,-0.505000,-0.022100,-0.013600,-0.050100,-0.016500,-0.446000,-0.009000,-0.056600,-0.308000,-0.521000,-0.010000,-0.022400,-0.748000,-0.021900,-0.014900,-0.023100,-0.431000,-0.523000,-0.583000,-0.611000,-0.034400,-0.069300,-0.030900,-0.142000,-0.007440,-0.018000,-0.020700,-0.016400,0.003440,-0.225000,-0.034900,-0.371000,-0.006850,-0.559000,-0.493000,-0.022700,-0.033900,-0.022100,-0.138000,0.009200,-0.003230,-0.010600,-0.022500,-0.044000,-0.210000,-0.015800,-0.4

In [ ]:
# dataset[ (dataset['ClassLabel']==1)].__len__()

In [ ]:
# dataset[ (dataset['ClassLabel']==0)].__len__()

In [ ]:
# dataset[ (dataset['ClassLabel']==0) & (dataset['sex'] == 1)].__len__()

In [ ]:
# dataset[ (dataset['ClassLabel']==0) & (dataset['sex'] == 2)].__len__()

In [ ]:
# y = dataset[dataset['ClassLabel']==1].groupby('sex')['protime'].count()
# print(y)
# x=['Females','Males']
# plt.bar(x,y)
# plt.title('Class1 wrt Gender')

In [ ]:
# y = dataset[dataset['ClassLabel']==0].groupby('age')['protime'].count()

# print(y)
# x=dataset[ (dataset['ClassLabel']==0)]['age'].unique()
# print(x)
# plt.bar(x,y)
# plt.title('Class0 wrt Age')
# plt.ylim([0,10])
# plt.xlim([20,80])

In [ ]:
# normalize_data = 1-((maximums-dataset)/rng)
# normalize_data

In [ ]:
# def visualize(X,y,thetas):
#     x_0 = X[np.where(y==0.0)]
#     x_1 = X[np.where(y==1.0)]
    
#     plt.scatter([x_0[:,1]],[x_0[:,2]],c ='b',label='y=0')
#     plt.scatter([x_1[:,1]],[x_1[:,2]],c ='r',label='y=1')
    
#     x1 = np.arange(0, 1, 0.1)
#     x2 = -(thetas[0,0] + thetas[0,1]*x1)/thetas[0,2]
#     plt.plot(x1, x2, c = 'k', label = 'reg line')
    
#     plt.xlabel('x1')
#     plt.ylabel('x2')
#     plt.legend()
    
#     plt.show()

In [5]:
#  1/(1+e^-a)
def logistc_function(thetas,X):
    #sygmoid
    return 1.0/(1+np.exp(-np.dot(X,thetas.T)))
    

In [6]:
def cost_func(thetas, X, y):
    log_func_value = logistc_function(thetas, X)
    step1 = y* np.log(log_func_value)
    step2 = (1 - y) * np.log(1 - log_func_value)
    return np.mean(-step1 - step2)

In [7]:
# return direction
def log_gradient(thetas, X, y): 
    calc_1 = logistc_function(thetas, X) - y.reshape(X.shape[0],1) 
    calc_f = np.dot(calc_1.T, X) 
    return calc_f

In [8]:
#thetas is random but will go to the global minima
def gradient_descent(X, y, thetas, learning_rate, convergance_criteria):
    cost = cost_func(thetas , X, y)
    change = 1
    iter_count = 1
    
    while(change > convergance_criteria):
        old_cost = cost
        thetas = thetas - (learning_rate * log_gradient(thetas, X, y))
        cost = cost_func(thetas, X, y)
        change = old_cost - cost
        iter_count += 1
        
    return thetas , iter_count
    

In [24]:
def cross_validation(train_X,train_Y,w,learning_rate,epsilon, length):
    Err_A=0
    lowest=1000;
    bestw=w;
    initw=w;
    for k in range(1,11):
        low= length*(k-1)
        up = length*k - 1
        w=initw
        validate_x=train_X[low:up,:]
        t_x=np.append(train_X[0:low,:],train_X[up:(length*10-1),:],axis=0)
        validate_y=train_Y[low:up,:]
        validate_y = validate_y[:,-1]
        t_y=np.append(train_Y[0:low,:],train_Y[up:(length*10-1),:],axis=0)
        t_y = t_y[:,-1]
        w ,count= gradient_descent(t_x,t_y,w,learning_rate,0.001)
        Xw = np.dot(validate_x,w.T)
        a = np.dot(validate_y.T,validate_y)
        b = 2 * np.dot(validate_y.T,Xw)
        c = np.dot(validate_x,w.T)
        d = np.dot(c.T,Xw)
        err = a - b + d
        # print("Err =" + str(err))
        y_pred = predict(w,validate_x)
        # y_pred = y_pred[:,None]
        score = Accu_eval(y_pred, validate_y, length)
        print("iteration: ",k)     
        print("Accuracy: " + str(score) + "%")
        Err_A += err
        if(err<lowest):
           lowest=err
           bestw=w
 
    Err_A = Err_A / 10
    return (Err_A,w)

In [10]:
def predict(thetas, X):
    prob = logistc_function(thetas,X)
    #dicision boundary
    predicted_value = np.where(prob > 0.5, 1, 0)
    return np.squeeze(predicted_value)

In [22]:
def Accu_eval(y_pred, y, length):
    corrects = np.sum(y == y_pred) / length * 100
    return corrects

In [25]:
#main

dataset = read_data()
np.random.shuffle(dataset)
normalized = normalize(dataset)
print("dataset shape", normalized.shape)
learning_rate =0.001
covergance_criteria = 0.001
# copy all the rows apart from last column
# X = normalized[:127,:-1]
X = normalized[:450,:-1]
# test_X = normalized[-15:,:-1]
# print("test_x",test_X.shape)
#add ones as first column
# print(X.shape)
#c.shape[1] 为第一维的长度，c.shape[0] 为第二维的长度。
# np.hstack将参数元组的元素数组按水平方向进行叠加
# test_X = np.hstack((np.matrix(np.ones(test_X.shape[0])).T,test_X))
X = np.hstack((np.matrix(np.ones(X.shape[0])).T,X))
# copy last column
# y = normalized[:127,-1]
y = normalized[:450,-1]
# test_y = normalized[-15:,-1]
length = 45
# print('y is ',y)
# print(y.shape)
# print(y.shape)
print("main: " + str(X.shape))
#init thetas just zeros
thetas = np.matrix(np.zeros(X.shape[1]))
#training the thetas
(final_thetas, iter_count) = gradient_descent(X,y,thetas,learning_rate,covergance_criteria)
# print(final_thetas, iter_count)
y_pred = predict(final_thetas,X)
# print(y_pred)
corrects = np.sum(y == y_pred)
total = y.__len__()
print("Gradient descent:")
print("corrects: ", corrects)
print("total", total)
print("Accuracy: ", corrects/total * 100)
print("10 folder cross validation:")
y=y[:,None]
err, final_thetas = cross_validation(X,y,thetas,learning_rate,covergance_criteria,length)
# visualize(X,y,final_thetas)

dataset shape (453, 65)
main: (450, 65)
Gradient descent:
corrects:  250
total 450
Accuracy:  55.55555555555556
10 folder cross validation:
iteration:  1
Accuracy: 64.44444444444444%
iteration:  2
Accuracy: 64.44444444444444%
iteration:  3
Accuracy: 77.77777777777779%
iteration:  4
Accuracy: 66.66666666666666%
iteration:  5
Accuracy: 57.77777777777777%
iteration:  6
Accuracy: 73.33333333333333%
iteration:  7
Accuracy: 77.77777777777779%
iteration:  8
Accuracy: 71.11111111111111%
iteration:  9
Accuracy: 64.44444444444444%
iteration:  10
Accuracy: 66.66666666666666%


In [ ]:
X

matrix([[1.        , 0.32394366, 1.        , ..., 0.44186047, 0.61      ,
         0.        ],
        [1.        , 0.6056338 , 0.        , ..., 0.3255814 , 0.61      ,
         0.        ],
        [1.        , 1.        , 0.        , ..., 0.44186047, 0.61      ,
         0.        ],
        ...,
        [1.        , 0.76056338, 0.        , ..., 0.46511628, 0.61      ,
         1.        ],
        [1.        , 0.64788732, 1.        , ..., 0.46511628, 0.48      ,
         1.        ],
        [1.        , 0.50704225, 0.        , ..., 0.23255814, 0.42      ,
         1.        ]])

In [ ]:
dataset

array([[30.,  2.,  1., ..., 61.,  1.,  1.],
       [50.,  1.,  1., ..., 61.,  1.,  1.],
       [78.,  1.,  2., ..., 61.,  1.,  1.],
       ...,
       [61.,  1.,  1., ..., 61.,  2.,  1.],
       [53.,  2.,  1., ..., 48.,  2.,  1.],
       [43.,  1.,  2., ..., 42.,  2.,  0.]])

In [ ]:
normalized = normalize(dataset)

In [ ]:
normalized

array([[0.32394366, 1.        , 0.        , ..., 0.61      , 0.        ,
        1.        ],
       [0.6056338 , 0.        , 0.        , ..., 0.61      , 0.        ,
        1.        ],
       [1.        , 0.        , 1.        , ..., 0.61      , 0.        ,
        1.        ],
       ...,
       [0.76056338, 0.        , 0.        , ..., 0.61      , 1.        ,
        1.        ],
       [0.64788732, 1.        , 0.        , ..., 0.48      , 1.        ,
        1.        ],
       [0.50704225, 0.        , 1.        , ..., 0.42      , 1.        ,
        0.        ]])

In [ ]:
normalized[-3:]

array([[0.76056338, 0.        , 0.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 1.        , 1.        ,
        0.        , 1.        , 1.        , 0.06493506, 0.18215613,
        0.00946372, 0.46511628, 0.61      , 1.        , 1.        ],
       [0.64788732, 1.        , 0.        , 1.        , 0.        ,
        1.        , 1.        , 1.        , 1.        , 0.        ,
        0.        , 1.        , 0.        , 0.15584416, 0.20446097,
        0.00788644, 0.46511628, 0.48      , 1.        , 1.        ],
       [0.50704225, 0.        , 1.        , 1.        , 0.        ,
        1.        , 1.        , 1.        , 1.        , 0.        ,
        0.        , 0.        , 1.        , 0.11688312, 0.27509294,
        0.00788644, 0.23255814, 0.42      , 1.        , 0.        ]])